In [6]:
import numpy as np
import pandas as pd

# Dropping Columns with the DataFrame Object 

In [2]:
!ls

Data Cleaning Tutorial - Real Python.ipynb
Datasets
follow_tutorial.ipynb


In [3]:
bookData = pd.read_csv('Datasets/BL-Flickr-Images-Book.csv')
bookData.head() #.head() shows first couple of rows of data, for initial assessment 

,Identifier,Edition Statement,Place of Publication,Date of Publication,Publisher,Title,Author,Contributors,Corporate Author,Corporate Contributors,Former owner,Engraver,Issuance type,Flickr URL,Shelfmarks
0,206,NaN,London,1879 [1878],S. Tinsley & Co.,Walter Forbes. [A novel.] By A. A,A. A.,"FORBES, Walter.",NaN,NaN,NaN,NaN,monographic,http://www.flickr.com/photos/britishlibrary/ta...,British Library HMNTS 12641.b.30.
1,216,NaN,London; Virtue & Yorston,1868,Virtue & Co.,All for Greed. [A novel. The dedication signed...,"A., A. A.","BLAZE DE BURY, Marie Pauline Rose - Baroness",NaN,NaN,NaN,NaN,monographic,http://www.flickr.com/photos/britishlibrary/ta...,British Library HMNTS 12626.cc.2.
2,218,NaN,London,1869,"Bradbury, Evans & Co.",Love the Avenger. By the author of “All for Gr...,"A., A. A.","BLAZE DE BURY, Marie Pauline Rose - Baroness",NaN,NaN,NaN,NaN,monographic,http://www.flickr.com/photos/britishlibrary/ta...,British Library HMNTS 12625.dd.1.
3,472,NaN,London,1851,James Darling,"Welsh Sketches, chiefly ecclesiastical, to the...","A., E. S.","Appleyard, Ernest Silvanus.",NaN,NaN,NaN,NaN,monographic,http://www.flickr.com/photos/britishlibrary/ta...,British Library HMNTS 10369.bbb.15.
4,480,"A new edition, revised, etc.",London,1857,Wertheim & Macintosh,"[The World in which I live, and my place in it...","A., E. S.","BROOME, John Henry.",NaN,NaN,NaN,NaN,monographic,http://www.flickr.com/photos/britishlibrary/ta...,British Library HMNTS 9007.d.28.


In [4]:
#we see a couple of unwanted columns for our particular task, so lets drop them.

#list containing all columns not relating to the books themselves
dropList = ['Edition Statement', 'Publisher', 'Contributors','Corporate Author','Corporate Contributors', 'Former owner', 'Engraver','Issuance type', 'Flickr URL', 'Shelfmarks']

#inplace as True and axis=1 tells pandas that we want changes made directly to the object 
bookData.drop(dropList, inplace=True, axis=1)

bookData.head()

,Identifier,Place of Publication,Date of Publication,Title,Author
0,206,London,1879 [1878],Walter Forbes. [A novel.] By A. A,A. A.
1,216,London; Virtue & Yorston,1868,All for Greed. [A novel. The dedication signed...,"A., A. A."
2,218,London,1869,Love the Avenger. By the author of “All for Gr...,"A., A. A."
3,472,London,1851,"Welsh Sketches, chiefly ecclesiastical, to the...","A., E. S."
4,480,London,1857,"[The World in which I live, and my place in it...","A., E. S."


In [ ]:
#another way to drop the columns is to pass the list directly to columns, this omits the axis=1 and is more intutive 

bookData.drop(columns=dropList, inplace=True)

bookData.head()

# Changing index of a data frame

In [5]:
# many times, it is useful to have a unique identifier column to be the index of your data

# being able to change this index improves labeling and slicing of our dataset

# we can check if a column is filled with unique values with: 

bookData['Identifier'].is_unique

True

In [6]:

# now that we have confirmed the uniqueness of the column, let's set the index to that column

bookData = bookData.set_index('Identifier')  
# we do this because by default .set_index() returns a modified version of the data frame
# to do it without this reassignment step we can use:
# bookData.set_index('Identifier', inplace=True)

bookData.head()

,Place of Publication,Date of Publication,Title,Author
Identifier,,,,
206,London,1879 [1878],Walter Forbes. [A novel.] By A. A,A. A.
216,London; Virtue & Yorston,1868,All for Greed. [A novel. The dedication signed...,"A., A. A."
218,London,1869,Love the Avenger. By the author of “All for Gr...,"A., A. A."
472,London,1851,"Welsh Sketches, chiefly ecclesiastical, to the...","A., E. S."
480,London,1857,"[The World in which I live, and my place in it...","A., E. S."


In [7]:
# now we should be able to select rows by identifier using .loc 

bookData.loc[206]

Place of Publication                               London
Date of Publication                           1879 [1878]
Title                   Walter Forbes. [A novel.] By A. A
Author                                              A. A.
Name: 206, dtype: object

## so previously we had our indeces from 0 like a python list, but now our indeces are values of 'Identifier'

# now let us tidy up some data in the columns

In [8]:
# see how many columns are "dtype: object" which essentially are strs, aka a mess of characters

bookData.dtypes.value_counts()

object    4
dtype: int64

In [9]:
# lets enforce numeric properties onto the publish date column

#check from id 1905, just the Date of Publication column
bookData.loc[1905:, 'Date of Publication'].head()

Identifier
1905           1888
1929    1839, 38-54
2836           1897
2854           1865
2956        1860-63
Name: Date of Publication, dtype: object

## this looks very messy, as one book should only have one date of publication 

In [10]:
# lets clean this up: 

# 1. remove extra dates in square brackets
# 2. remove uncertain dates and replace with NaN value from numpy
# 3. convert date ranges to start date
# 4. convert nan strings to NaN counter part

# this is when regex comes in handy 
# we just need to find any 4 digits at the start of a string

# the r in front tells python it is a raw string (backslash is no longer an escape character), \d means digit, 
# {4} means repeat 4 times, and () groups \d and {4} to create 4 digits, ^ means search from the start

regex = r'^(\d{4})'

extracted = bookData['Date of Publication'].str.extract(regex, expand=False)

extracted.head()

Identifier
206    1879
216    1868
218    1869
472    1851
480    1857
Name: Date of Publication, dtype: object

In [11]:
# so now we see much nicer strings in the Date of Publication column ( which we saved into extracted, original
# is still messy as seen below )

bookData['Date of Publication'].head()

Identifier
206    1879 [1878]
216           1868
218           1869
472           1851
480           1857
Name: Date of Publication, dtype: object

In [12]:
# great, now lets change all the data types in extracted to float 

extracted = pd.to_numeric(extracted)
extracted.head()

Identifier
206    1879.0
216    1868.0
218    1869.0
472    1851.0
480    1857.0
Name: Date of Publication, dtype: float64

In [13]:

# now that everything is good, lets replace the Date of Publication column values with those from extracted

bookData['Date of Publication'] = extracted

bookData['Date of Publication'].dtype

dtype('float64')

In [14]:

# now we are able to do numeric operations on our column (below operation shows that 10% of data is null values):

bookData['Date of Publication'].isnull().sum() / len(bookData)

0.11717147339205986

In [15]:
# average Date of publication: 

bookData['Date of Publication'].sum() / len(bookData)

1639.0848316640522


# we can combine str methods and numpy to clean columns more effectively

In [16]:
# data frame object have the .str attribute. This allows for string operations in pandas to mimic native python 
# string methods like .split() and .capitalize()

# np.where(condition, [x,y]) returns either x or y, depending on condition
# this can also be nested to create a compound if, then statement allowing us to computer for multiple conditions
# lets use this to clean the Place of Publication column. 

# first lets see the contents of the Place of Publication column. 

bookData['Place of Publication'].head()

Identifier
206                      London
216    London; Virtue & Yorston
218                      London
472                      London
480                      London
Name: Place of Publication, dtype: object

In [17]:
# the tutorial showcases two specific cases: 

bookData.loc[4157862]

Place of Publication                                  Newcastle-upon-Tyne
Date of Publication                                                  1867
Title                   Local Records; or, Historical Register of rema...
Author                      FORDYCE, T. - Printer, of Newcastle-upon-Tyne
Name: 4157862, dtype: object

In [18]:
bookData.loc[4159587]

Place of Publication                                  Newcastle upon Tyne
Date of Publication                                                  1834
Title                   An historical, topographical and descriptive v...
Author                                              Mackenzie, E. (Eneas)
Name: 4159587, dtype: object

In [19]:
# these two books have the same Place of Publication, but one has hypens while one does not. 
# we can clean these up with str.contains() to create a boolean map of which rows have a certain value

regex2 = r'Newcastle.upon.Tyne'

pub = bookData['Place of Publication']

newCastle = pub.str.contains(regex2)

# as we can see, the lines containing the words have the boolean map of True
print(newCastle[4159587])
print(newCastle[4157862])

newCastle[:5]

True
True


Identifier
206    False
216    False
218    False
472    False
480    False
Name: Place of Publication, dtype: bool

In [20]:
# lets create some more conditions
regex3 = 'London'

london = pub.str.contains(regex3)


# we combine them with np.where() here I nested them to fix London, Newcastle, and all hypen issues 

bookData['Place of Publication'] = np.where(newCastle, 'Newcastle upon Tyne', 
                                            np.where(london, 'London', pub.str.replace('_', ' ')))

print(bookData.loc[4157862])
bookData['Place of Publication'].head()

Place of Publication                                  Newcastle upon Tyne
Date of Publication                                                  1867
Title                   Local Records; or, Historical Register of rema...
Author                      FORDYCE, T. - Printer, of Newcastle-upon-Tyne
Name: 4157862, dtype: object


Identifier
206    London
216    London
218    London
472    London
480    London
Name: Place of Publication, dtype: object

In [21]:

# now the rows are a lot cleaner for Place of Publication
bookData.head()

,Place of Publication,Date of Publication,Title,Author
Identifier,,,,
206,London,1879.0,Walter Forbes. [A novel.] By A. A,A. A.
216,London,1868.0,All for Greed. [A novel. The dedication signed...,"A., A. A."
218,London,1869.0,Love the Avenger. By the author of “All for Gr...,"A., A. A."
472,London,1851.0,"Welsh Sketches, chiefly ecclesiastical, to the...","A., E. S."
480,London,1857.0,"[The World in which I live, and my place in it...","A., E. S."


# Cleaning entire data set with appymap function

In [1]:

# now we are going to use the university towns dataset
!ls

Data Cleaning Tutorial - Real Python.ipynb
Datasets
follow_tutorial.ipynb


In [4]:
!cd Datasets && ls

BL-Flickr-Images-Book.csv olympics.csv              university_towns.txt


In [9]:
# since this is a txt file, we will read it in a different way. 

!head Datasets/university_towns.txt

Alabama[edit]
Auburn (Auburn University)[1]
Florence (University of North Alabama)
Jacksonville (Jacksonville State University)[2]
Livingston (University of West Alabama)[2]
Montevallo (University of Montevallo)[2]
Troy (Troy University)[2]
Tuscaloosa (University of Alabama, Stillman College, Shelton State)[3][4]
Tuskegee (Tuskegee University)[5]
Alaska[edit]


In [16]:

# so here we see state names wrapped by university towns in that state, and that each state is 
# indicated with "[edit]"

# we can take advantage of this and create a tuple of (state, city) and create a pandas dataframe with it 

uni_towns = []

# loop through each line and check if [edit] is found, if it is then we know that it is a state else we append tuple 
with open('Datasets/university_towns.txt') as file:
    for line in file:
        if "[edit]" in line:
            state = line
        
        else:
            uni_towns.append((state, line))

uni_towns[:5]

[('Alabama[edit]\n', 'Auburn (Auburn University)[1]\n'),
 ('Alabama[edit]\n', 'Florence (University of North Alabama)\n'),
 ('Alabama[edit]\n', 'Jacksonville (Jacksonville State University)[2]\n'),
 ('Alabama[edit]\n', 'Livingston (University of West Alabama)[2]\n'),
 ('Alabama[edit]\n', 'Montevallo (University of Montevallo)[2]\n')]

In [19]:
# now with this list of tuples, we can wrap it in a pandas dataFrame with state as the column 

townData = pd.DataFrame(uni_towns, columns = ['States', 'City name'])

townData.head()


,States,City name
0,Alabama[edit]\n,Auburn (Auburn University)[1]\n
1,Alabama[edit]\n,Florence (University of North Alabama)\n
2,Alabama[edit]\n,Jacksonville (Jacksonville State University)[2]\n
3,Alabama[edit]\n,Livingston (University of West Alabama)[2]\n
4,Alabama[edit]\n,Montevallo (University of Montevallo)[2]\n


In [23]:
# the reason I have not cleaned the strings from the loop above is because pandas makes cleaning strings much easier
# apply map applies a function to each element individually

# let's first create a function which gets rid of elements contained in round and square brackets
# the function returns a new, modified item 


def get_city (item):
    if ' (' in item:
        return item[:item.find(' (')]
    
    elif '[' in item:
        return item[:item.find('[')]
    
    else:
        return item
    
    
# now that we have this function we can apply it to each element in townData

townData = townData.applymap(get_city)

townData.head()

# now it looks a lot cleaner. Apply map is an useful feature which passes every element of the dataframe to a function
# the original value in the dataframe is replaced with the returned value from the function

,States,City name
0,Alabama,Auburn
1,Alabama,Florence
2,Alabama,Jacksonville
3,Alabama,Livingston
4,Alabama,Montevallo


# Renaming columns and skipping rows

In [24]:
# often, the datasets we work with will have unintuitive names and/ or rows that have no useful information at the 
# start or end

# to demonstrate this we will be working the with the olympics dataset

olympData = pd.read_csv('Datasets/olympics.csv')

olympData.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,NaN,? Summer,01 !,02 !,03 !,Total,? Winter,01 !,02 !,03 !,Total,? Games,01 !,02 !,03 !,Combined total
1,Afghanistan (AFG),13,0,0,2,2,0,0,0,0,0,13,0,0,2,2
2,Algeria (ALG),12,5,2,8,15,3,0,0,0,0,15,5,2,8,15
3,Argentina (ARG),23,18,24,28,70,18,0,0,0,0,41,18,24,28,70
4,Armenia (ARM),5,1,2,9,12,6,0,0,0,0,11,1,2,9,12


In [26]:
# wow these column names are awful. Let's change them so it is easier to understand. 

# the columns are indeces, the first row contains actual column names but those are pretty bad too
# this means we need to skip a row, and set the second row as column names, and rename them

# we can directly skip rows when reading them by passing some parameters to pd.read_csv

olympData = pd.read_csv('Datasets/olympics.csv', header=1)
olympData.head()

,Unnamed: 0,? Summer,01 !,02 !,03 !,Total,? Winter,01 !.1,02 !.1,03 !.1,Total.1,? Games,01 !.2,02 !.2,03 !.2,Combined total
0,Afghanistan (AFG),13,0,0,2,2,0,0,0,0,0,13,0,0,2,2
1,Algeria (ALG),12,5,2,8,15,3,0,0,0,0,15,5,2,8,15
2,Argentina (ARG),23,18,24,28,70,18,0,0,0,0,41,18,24,28,70
3,Armenia (ARM),5,1,2,9,12,6,0,0,0,0,11,1,2,9,12
4,Australasia (ANZ) [ANZ],2,3,4,5,12,0,0,0,0,0,2,3,4,5,12


In [30]:
# now we read the header from the second row instead of the first

# to rename the columns we can use pandas' rename() method, which takes a dict

new_labels = {
    'Unnamed: 0': 'Country',
    '? Summer': 'Summer Olympics',
    '01 !': 'Gold',
    '02 !': 'Silver',
    '03 !': 'Bronze',
    '? Winter': 'Winter Olympics',
    '01 !.1': 'Gold',
    '02 !.1': 'Silver',
    '03 !.1': 'Bronze',
    '01 !.2': 'Gold',
    '02 !.2': 'Silver',
    '03 !.2': 'Bronze',
    '? Games': '# Games'
    
}
    
# recall that setting inplace to True makes changes directly to the object
olympData.rename(columns=new_labels, inplace=True)

olympData.head()

,Country,Summer Olympics,Gold,Silver,Bronze,Total,Winter Olympics,Gold,Silver,Bronze,Total.1,# Games,Gold,Silver,Bronze,Combined total
0,Afghanistan (AFG),13,0,0,2,2,0,0,0,0,0,13,0,0,2,2
1,Algeria (ALG),12,5,2,8,15,3,0,0,0,0,15,5,2,8,15
2,Argentina (ARG),23,18,24,28,70,18,0,0,0,0,41,18,24,28,70
3,Armenia (ARM),5,1,2,9,12,6,0,0,0,0,11,1,2,9,12
4,Australasia (ANZ) [ANZ],2,3,4,5,12,0,0,0,0,0,2,3,4,5,12


In [31]:
# now this is the end of the tutorial

# but lets clean the columns with what we've learned above:

# lets clean the country column

olympData['Country']

0                                 Afghanistan (AFG)
1                                     Algeria (ALG)
2                                   Argentina (ARG)
3                                     Armenia (ARM)
4                           Australasia (ANZ) [ANZ]
                           ...                     
142    Independent Olympic Participants (IOP) [IOP]
143                              Zambia (ZAM) [ZAM]
144                            Zimbabwe (ZIM) [ZIM]
145                          Mixed team (ZZX) [ZZX]
146                                          Totals
Name: Country, Length: 147, dtype: object

In [33]:
# lets get rid of the [] names, since they look like they are redundant
# since it is only one column and not the whole dataset, we can use str.contains and np.where
# but first let's check if apply map works on single columns

country = olympData['Country']

country = country.applymap(get_city)


AttributeError: 'Series' object has no attribute 'applymap'

In [35]:
# since that doesn't work, lets us our friend regex

regex = r'\['

country = olympData['Country']

countryMap = country.str.contains(regex)

countryMap

0      False
1      False
2      False
3      False
4       True
       ...  
142     True
143     True
144     True
145     True
146    False
Name: Country, Length: 147, dtype: bool

In [42]:
# now that we have the boolean map, let's iterate through and get rid of the [
# well it turns out np.where isn't very useful since we can't access each element to make changes

index = 0
for element in country:
    if '[' in element: 
        country[index] = element[:element.find('[')]
    
    index += 1
    
country



//anaconda3/envs/dataScience/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3326: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exec(code_obj, self.user_global_ns, self.user_ns)
//anaconda3/envs/dataScience/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


0                            Afghanistan (AFG)
1                                Algeria (ALG)
2                              Argentina (ARG)
3                                Armenia (ARM)
4                           Australasia (ANZ) 
                        ...                   
142    Independent Olympic Participants (IOP) 
143                              Zambia (ZAM) 
144                            Zimbabwe (ZIM) 
145                          Mixed team (ZZX) 
146                                     Totals
Name: Country, Length: 147, dtype: object

In [44]:
# hmm I get a warning but it looks like it works right now 

olympData['Country'] = country
olympData.head()

,Country,Summer Olympics,Gold,Silver,Bronze,Total,Winter Olympics,Gold,Silver,Bronze,Total.1,# Games,Gold,Silver,Bronze,Combined total
0,Afghanistan (AFG),13,0,0,2,2,0,0,0,0,0,13,0,0,2,2
1,Algeria (ALG),12,5,2,8,15,3,0,0,0,0,15,5,2,8,15
2,Argentina (ARG),23,18,24,28,70,18,0,0,0,0,41,18,24,28,70
3,Armenia (ARM),5,1,2,9,12,6,0,0,0,0,11,1,2,9,12
4,Australasia (ANZ),2,3,4,5,12,0,0,0,0,0,2,3,4,5,12
